## Dependencies

In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# MALARIA DETECTION

In [2]:
import tensorflow as tf
import numpy as np
import gradio as gr
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define paths
BINARY_MODEL_PATH = '/content/drive/MyDrive/deep_binary_classification_model.h5'
MULTICLASS_MODEL_PATH = '/content/drive/MyDrive/transfer_learning_model.h5'

# Load models
IMG_SIZE = (128, 128)
binary_model = tf.keras.models.load_model(BINARY_MODEL_PATH)
multiclass_model = tf.keras.models.load_model(MULTICLASS_MODEL_PATH)

multiclass_labels = [
    "falciparum_gametocyte", "falciparum_ring", "falciparum_schizont", "falciparum_trophozoite",
    "malariae_gametocyte", "malariae_ring", "malariae_schizont", "malariae_trophozoite",
    "ovale_gametocyte", "ovale_ring", "ovale_schizont", "ovale_trophozoite",
    "vivax_gametocyte", "vivax_ring", "vivax_schizont", "vivax_trophozoite"
]


def preprocess_image(image):
    img = load_img(image, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_image(image):
    img_array = preprocess_image(image)
    binary_prediction = binary_model.predict(img_array)
    if binary_prediction[0][0] > 0.5:
        binary_result = "Uninfected"
        binary_confidence = binary_prediction[0][0]
        return binary_result, None, binary_confidence
    else:
        binary_result = "Parasitized"
        binary_confidence = 1 - binary_prediction[0][0]
        multiclass_prediction = multiclass_model.predict(img_array)
        predicted_class = np.argmax(multiclass_prediction, axis=1)[0]
        confidence = multiclass_prediction[0][predicted_class]
        subclass = multiclass_labels[predicted_class]
        return binary_result, subclass, confidence


In [3]:
def interface_predict(image):
    binary_result, subclass, confidence = predict_image(image)

    if binary_result == "Uninfected":
        result = f"Prediction: {binary_result}"
        subclass_info = "No subtype applicable."
    else:
        result = f"Prediction: {binary_result}"
        subclass_info = f"Subtype: {subclass}"

    return result, subclass_info


# Define Gradio Blocks
with gr.Blocks() as demo:
    gr.Markdown("# Malaria Detection System")
    gr.Markdown("Upload an image to determine if it shows a malaria infection and identify the subtype if applicable.")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="filepath", label="Upload Image")
            predict_button = gr.Button("Predict")
        with gr.Column():
            prediction_output = gr.Textbox(label="Prediction", lines=2)
            subclass_output = gr.Textbox(label="Subtype (if infected)", lines=2)

    predict_button.click(
        fn=interface_predict,
        inputs=image_input,
        outputs=[prediction_output, subclass_output]
    )


    gr.Markdown("### Note: Please ensure the image is clear for accurate predictions.")

# Launch the interface
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1362832b1451662213.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
